In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [4]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "lypguANrlocwapeIEhWYKKhWM"
    consumer_secret = "ltarjR26uAQ7ZdV858GK5tbzNxDQuTb4eKTDef24lhHgcYK8XD"
    access_token = "1516939692708548608-7JQBSsV54rjKjX8TBuWGhls9LBF7oe"
    access_token_secret = "dzDKfcC34wThU0zb1c13Vru5fB4Fny8RG9atFGD7bYevS"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.521822, 45.661688, -117.032350, 48.999213,    # WA
                 -73.473629, 41.042289, -69.882444, 42.632089]      # MA, CT, RI

    stream_listener = StreamListener(time_limit=1200, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517214780658012161, 'username': 'JahJukeMPMG', 'created_at': 'Thu Apr 21 18:52:46 +0000 2022', 'lng': -122.3308, 'lat': 47.6046, 'text': 'Just posted a photo @ Seattle, Washington https://t.co/sTskeIHZdn'}
{'id': 1517214811729240064, 'username': 'ArturoEdvardsen', 'created_at': 'Thu Apr 21 18:52:54 +0000 2022', 'lng': -122.149768, 'lat': 47.5978565, 'text': '@elonmusk Would love to work for one of your companies Elon.'}
{'id': 1517214833594167296, 'username': 'NotThatChickNan', 'created_at': 'Thu Apr 21 18:52:59 +0000 2022', 'lng': -120.882482, 'lat': 47.273022, 'text': '@Currtis25Harr *sitting with my husband and getting mad and I’m like wow the audacity right here it’s so much I’m actually angry'}
{'id': 1517214928481923072, 'username': 'ArturoEdvardsen', 'created_at': 'Thu Apr 21 18:53:22 +0000 2022', 'lng': -122.149768, 'lat': 47.5978565, 'text': '@RetroTechNoir Hell yes.'}
{'id': 1517214992763809793, 'username': 'ParaguayBeauty', 'created_at': 'Thu Apr 21 18:53:37 +0000 20

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
